**Question 3 -**

Train a Pure CNN with less than 10000 trainable parameters using the MNIST
Dataset having minimum validation accuracy of 99.40%

Note -
1. Code comments should be given for proper code understanding.
2. Implement in both PyTorch and Tensorflow respectively


In [ ]:
# Importing libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Device configuration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyperparameters

num_epochs = 10
batch_size = 128
learning_rate = 0.001

In [ ]:
# Dataset and Data Loader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 9912422/9912422 [00:00<00:00, 102637323.66it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 14305112.64it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 26418312.03it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4263770.99it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Pure CNN
class PureCNN(nn.Module):
    def __init__(self):
        super(PureCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
model = PureCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training the model

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))


Epoch [1/10], Step [100/469], Loss: 0.2530
Epoch [1/10], Step [200/469], Loss: 0.2255
Epoch [1/10], Step [300/469], Loss: 0.1639
Epoch [1/10], Step [400/469], Loss: 0.1587
Epoch [2/10], Step [100/469], Loss: 0.2020
Epoch [2/10], Step [200/469], Loss: 0.0442
Epoch [2/10], Step [300/469], Loss: 0.0928
Epoch [2/10], Step [400/469], Loss: 0.0550
Epoch [3/10], Step [100/469], Loss: 0.1071
Epoch [3/10], Step [200/469], Loss: 0.0164
Epoch [3/10], Step [300/469], Loss: 0.0640
Epoch [3/10], Step [400/469], Loss: 0.0132
Epoch [4/10], Step [100/469], Loss: 0.0481
Epoch [4/10], Step [200/469], Loss: 0.0201
Epoch [4/10], Step [300/469], Loss: 0.0359
Epoch [4/10], Step [400/469], Loss: 0.0898
Epoch [5/10], Step [100/469], Loss: 0.0631
Epoch [5/10], Step [200/469], Loss: 0.0112
Epoch [5/10], Step [300/469], Loss: 0.0138
Epoch [5/10], Step [400/469], Loss: 0.0180
Epoch [6/10], Step [100/469], Loss: 0.0239
Epoch [6/10], Step [200/469], Loss: 0.0687
Epoch [6/10], Step [300/469], Loss: 0.0713
Epoch [6/10

In [ ]:
# Evaluation on test data

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Test Accuracy: {:.2f}%'.format(accuracy))

Test Accuracy: 98.52%
